In [1]:
import muon as mu
path = "/data/tmpA/andrem/celloracle/out/AML12_DX_with_atac/mdata.h5mu"
mdata = mu.read_h5mu(path)

/home/andrem/miniconda3/envs/benchmark/lib/python3.10/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/andrem/miniconda3/envs/benchmark/lib/python3.10/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [5]:
mdata["rna"].obs_names

Index(['AAACCCAAGGGTTAAT-1___AML12_DX', 'AAACCCAAGTTGCCTA-1___AML12_DX',
       'AAACCCACAAATGGTA-1___AML12_DX', 'AAACCCACAACACTAC-1___AML12_DX',
       'AAACCCACACAACCGC-1___AML12_DX', 'AAACCCACACGACGTC-1___AML12_DX',
       'AAACCCACATTCTGTT-1___AML12_DX', 'AAACCCAGTAACGTTC-1___AML12_DX',
       'AAACCCAGTCTCAGGC-1___AML12_DX', 'AAACCCATCGAGTGAG-1___AML12_DX',
       ...
       'TTTGGTTGTCAAAGAT-1___AML12_DX', 'TTTGGTTGTGGGCTCT-1___AML12_DX',
       'TTTGGTTGTTAGGAGC-1___AML12_DX', 'TTTGTTGAGACCGTTT-1___AML12_DX',
       'TTTGTTGAGAGGCCAT-1___AML12_DX', 'TTTGTTGAGTAGGCCA-1___AML12_DX',
       'TTTGTTGGTAGATTGA-1___AML12_DX', 'TTTGTTGGTGCAGATG-1___AML12_DX',
       'TTTGTTGGTTGTGTTG-1___AML12_DX', 'TTTGTTGTCCGTTGGG-1___AML12_DX'],
      dtype='object', length=6826)

In [8]:
mdata["atac"].obs_names

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '6816', '6817', '6818', '6819', '6820', '6821', '6822', '6823', '6824',
       '6825'],
      dtype='object', length=6826)